# 01 Data Preparation

Load and clean the OSM-derived grid data, then package it for remote runs.


In [187]:
from pathlib import Path
import sys
import os
import pypsa
import numpy as np

def find_repo_root(max_up=6):
    p = Path.cwd().resolve()
    for _ in range(max_up):
        if (p / 'README.md').exists() or (p / '.git').exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    return Path.cwd().resolve()

repo_root = find_repo_root()
src_path = repo_root / 'src/'
if str(src_path) not in sys.path:
    sys.path.insert(1, str(src_path))
print(f"Using src path: {src_path}")
print(f"Repository root: {repo_root}")

import pypsa_simplified as ps

Using src path: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model/src
Repository root: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model


## Data sources and parsing
- **OSM prebuilt electricity network** (`data/raw/OSM Prebuilt Electricity Network/`): buses, lines, links, converters, transformers.
- **Custom CSV parsing**: `prepare_osm_source` uses a geometry-safe loader (handles commas inside WKT) to keep column counts correct.
- **Endpoint extraction**: First/last coordinates are pulled from WKT to map line/link endpoints to buses (tolerance 1e-5 degrees).
- **Country filter**: Defaults to DE/FR/PL/AT/IT; adjust via `countries` if needed.


In [188]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")
import pandas as pd
import tzdata
from pypsa_simplified import prepare_osm_source
from pypsa_simplified import prepare_generator_data


osm_dir = repo_root / "data" / "raw" / "OSM Prebuilt Electricity Network"
sources = prepare_osm_source(osm_dir)
print({k: v.shape if hasattr(v, 'shape') else v for k, v in sources.items()})

RawData = ps.data_prep.RawData(sources)

source = prepare_generator_data(repo_root / "data" / "raw" / "powerplants.csv")

RawData.data['generators'] = source

{'buses': (6737, 10), 'lines': (8994, 16), 'converters': (67, 7), 'links': (38, 10), 'transformers': (875, 8), 'generators': None, 'loads': None, 'storage': None}


In [189]:
RawData.data['buses']

,bus_id,voltage,dc,symbol,under_construction,tags,x,y,country,geometry
0,AL1-220,220.0,f,Substation,f,AL1,20.111702,42.096798,AL,POINT (20.111702399999995 42.09679778825765)
1,AL10-220,220.0,f,Substation,f,AL10,19.522149,40.478444,AL,POINT (19.522149214338864 40.47844442711058)
2,AL2-220,220.0,f,Substation,f,AL2,20.031168,42.069050,AL,POINT (20.031168199999993 42.06905038823308)
3,AL3-400,400.0,f,Substation,f,AL3,19.652495,42.010831,AL,POINT (19.652494799999996 42.01083068818158)
4,AL4-220,220.0,f,Substation,f,AL4,19.972850,41.584771,AL,POINT (19.9728498 41.58477058780021)
...,...,...,...,...,...,...,...,...,...,...
6732,way/99694910-380,380.0,f,Substation,f,way/99694910,7.789278,44.462732,IT,POINT (7.7892782005302115 44.46273179679329)
6733,way/99722046-225,225.0,f,Substation,f,way/99722046,-0.114171,47.206984,FR,POINT (-0.1141712110057602 47.206983706128305)
6734,way/99722046-400,400.0,f,Substation,f,way/99722046,-0.113930,47.207199,FR,POINT (-0.113930394824857 47.20719863601106)
6735,way/99826025-220,220.0,f,Substation,f,way/99826025,7.785444,44.875461,IT,POINT (7.785443587366926 44.87546070934813)


In [190]:
EU27 = [
    'AT', 'BE', 'BG', 'HR', 'CZ', 'DK', 'EE', 'EL', 'FI', 'FR',
    'DE', 'GR', 'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL',
    'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE'
] + ['UK', 'CH', 'NO', 'GB']  # include UK, CH, NO for practical purposes

In [220]:
v_startswith = np.vectorize(lambda x, prefix: str(x).startswith(prefix))
v_contains = np.vectorize(lambda x, substring: substring in str(x))
v_first_n = np.vectorize(lambda x, n: str(x)[:n])
v_and = np.vectorize(lambda x, y: x and y)
v_isbool = np.vectorize(lambda x,y: x==y)

load_data_path = repo_root / "data" / "raw" / "time_series_60min_singleindex_filtered.csv"

# Load the load data
load_data = pd.read_csv(load_data_path, index_col=0, parse_dates=True)
# Column 'utc_timestamp' to datetime index and in GMT+1
load_data.index = pd.to_datetime(load_data.index).tz_convert(60*60)


a = np.array(load_data.columns.tolist())

countries = v_first_n(a, 2)
countries = countries[countries != 'GB']
countries = np.unique(countries)
cols = load_data.columns

load_cols = [f"{country}_load_actual_entsoe_transparency" if country in EU27 else "not included" for country in countries]

mask = np.where(v_isbool(v_startswith(load_cols, "not").tolist(), False).tolist())[0].tolist()
new_load_cols = []
for i in mask:
    new_load_cols.append(load_cols[i])

new_load_cols

a = a[v_startswith(a, "GB")]
a = a[v_contains(a, "_load_actual")]

new_load_cols = new_load_cols + a.tolist()


In [ ]:
load_data = load_data[new_load_cols]
display(load_data.head())


,AT_load_actual_entsoe_transparency,BE_load_actual_entsoe_transparency,BG_load_actual_entsoe_transparency,CH_load_actual_entsoe_transparency,CZ_load_actual_entsoe_transparency,DE_load_actual_entsoe_transparency,DK_load_actual_entsoe_transparency,EE_load_actual_entsoe_transparency,ES_load_actual_entsoe_transparency,FI_load_actual_entsoe_transparency,...,NO_load_actual_entsoe_transparency,PL_load_actual_entsoe_transparency,PT_load_actual_entsoe_transparency,RO_load_actual_entsoe_transparency,SE_load_actual_entsoe_transparency,SI_load_actual_entsoe_transparency,SK_load_actual_entsoe_transparency,GB_GBN_load_actual_entsoe_transparency,GB_NIR_load_actual_entsoe_transparency,GB_UKM_load_actual_entsoe_transparency
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 01:00:00+01:00,5946.0,9484.0,NaN,NaN,NaN,41151.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26758.0,849.0,27607.0
2015-01-01 02:00:00+01:00,5726.0,9152.0,4585.0,7640.14,5957.57,40135.0,3100.02,764.7,22734.0,8735.4,...,15104.99,13979.46,5123.9,NaN,14485.0,1045.47,2728.0,27166.0,782.0,27948.0
2015-01-01 03:00:00+01:00,5347.0,8799.0,4440.0,7391.44,5878.87,39106.0,2980.39,749.8,21286.0,8626.4,...,14882.56,13453.78,4771.1,NaN,14298.0,1004.79,2626.0,24472.0,738.0,25210.0
2015-01-01 04:00:00+01:00,5249.0,8567.0,4418.0,7332.57,5827.80,38765.0,2933.49,746.5,20264.0,8674.1,...,14755.09,13306.00,4443.5,NaN,14249.0,983.79,2618.0,23003.0,690.0,23693.0


In [ ]:
load_data.columns = v_first_n(load_data.columns,2)

array(['AT', 'BE', 'BG', 'CH', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR',
       'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'NL', 'NO', 'PL',
       'PT', 'RO', 'SE', 'SI', 'SK', 'GB', 'GB', 'GB'], dtype='<U2')

In [217]:
temp = pd.Series(load_data['AT_load_actual_entsoe_transparency'].values, index=load_data.index)
temp2 = pd.Series(load_data['BE_load_actual_entsoe_transparency'].values, index=load_data.index)

## Serialize for remote processing
The serialized artifact is compact (gzip + pickle) and ready to `scp` to the server.


In [ ]:
# Save osm data source inputs
written = RawData.save(output_path=repo_root / "data" / "processed" / "osm_source_data.pkl.gz")
print(f"RawData.save wrote: {written}")

RawData.save wrote: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model/data/processed/osm_source_data.pkl.gz


In [ ]:
RawData = ps.data_prep.RawData(None)
load_path = repo_root / "data" / "processed" / "osm_source_data.pkl.pkl.gz"
RawData.load(input_path=written)
print(f"Loaded RawData from: {written}")

Loaded RawData from: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model/data/processed/osm_source_data.pkl.gz


In [7]:
# Load parquet from:
parquet_path = repo_root / "data" / "processed" / "jrc_population_nonzero.parquet"
# as pandas dataframe
jrc_pop = pd.read_parquet(parquet_path)
print(f"Loaded JRC population data from: {parquet_path}, shape: {jrc_pop.shape}")

Loaded JRC population data from: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model/data/processed/jrc_population_nonzero.parquet, shape: (34363796, 3)


In [8]:
# Sort by lat and then lon
jrc_pop = jrc_pop.sort_values(by=['lat', 'lon']).reset_index(drop=True)
jrc_pop.head()

,lon,lat,population
0,-17.979866,27.639805,7
1,-17.982065,27.640021,5
2,-17.978908,27.640127,16
3,-17.984263,27.640237,30
4,-17.983305,27.640559,13


## Create a network
Use the RawData to create the first network

In [ ]:
snapshots = pd.date_range("2024-01-01", "2024-12-31 23:00", freq="h")
NETWORK_METADATA = {
    'name': 'Simplified European Electricity Network',
    'snapshots': snapshots,
    'countries': None,
    'generation_carriers': None,
    'transmission_carriers': None,
}

In [ ]:
n = pypsa.Network()
n = ps.build_network(n, RawData, options=NETWORK_METADATA)



### Next
- Use `notebooks/main.ipynb` to transfer the artifact and trigger the remote optimization.
- For custom country lists or tolerance, pass `countries`/`tol` to `prepare_osm_source`.
